# Purpose

# Imports

In [33]:
import autoreload
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import logging
logger = logging.getLogger()

import os
#os.chdir("work/python/notebooks") # go up to python/ level

import datetime as dt
import pytz

from io import BytesIO

import pickle
from googleapiclient.discovery import build

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Constants

In [37]:
# Token for plotly express mapbox viz
mapbox_token = open("../../secure_keys/public.mapbox_token").read()
px.set_mapbox_access_token(mapbox_token)

# Setup time stuff in consistent manner
TIME_FORMAT='%Y-%m-%d_%H:%M:%S'
TIMEZONE = pytz.timezone("UTC")

# Get logging formatted
LOG_FORMAT = '%(asctime)s: %(levelname)s (%(name)s) - %(message)s'

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(
    format=LOG_FORMAT,
    level=logging.INFO,
    datefmt='%m/%d/%Y %H:%M:%S')

# SBA Paycheck Protection Program Data



## Load Latest Cleaned File

Other members of the volunteer data scientist team are doing data cleaning of their own and pushing updates into a shared Google Drive, so let's get that first!

In [63]:
# Pull in our Google Drive creds
with open('secure_keys/token.pickle', 'rb') as token:
    creds = pickle.load(token)
    
service = build('drive', 'v3', credentials=creds)

In [82]:
# Find the info for All Data by State folder that contains raw PPP data
results = service.files().list(
    #fields="nextPageToken, files(id, name)",
q = "name = 'All Data by State'"
).execute()
data_folder_info = results.get('files', [])[0] # just access results dict with 'files' key or returns [] if key not there
data_folder_info

{'kind': 'drive#file',
 'id': '1uuO0075wElCtZVqk9HaUWPCxizohc0L9',
 'name': 'All Data by State',
 'mimeType': 'application/vnd.google-apps.folder'}

In [73]:
# Find the child folders in the All Data by State folder
results = service.files().list(
q = f"'{data_folder_info['id']}' in parents and mimeType = 'application/vnd.google-apps.folder'").execute()
data_subfolders = results.get('files', []) # just access results dict with 'files' key or returns [] if key not there
data_subfolders

[{'kind': 'drive#file',
  'id': '1g92YGs6BD40S89CqRZhlL2XXPQE-JIR8',
  'name': 'Wyoming',
  'mimeType': 'application/vnd.google-apps.folder'},
 {'kind': 'drive#file',
  'id': '19OOYSCuzi2jGJXUBMb8EDtfUxjRW8NyN',
  'name': 'Vermont',
  'mimeType': 'application/vnd.google-apps.folder'},
 {'kind': 'drive#file',
  'id': '1LCRqy05NV8nwaOg-YJOnt0ueow0WFJNq',
  'name': 'Virginia',
  'mimeType': 'application/vnd.google-apps.folder'},
 {'kind': 'drive#file',
  'id': '1XVK0rzn9DteU5Jg5rMootujPKzyfP5bS',
  'name': 'Wisconsin',
  'mimeType': 'application/vnd.google-apps.folder'},
 {'kind': 'drive#file',
  'id': '1NJNG2ewGUrela4JCNZOeq-rCs5WNX-RT',
  'name': 'Washington',
  'mimeType': 'application/vnd.google-apps.folder'},
 {'kind': 'drive#file',
  'id': '1rUua0BiZFioslEY-LJJMXglGIdFMzGly',
  'name': 'Virgin Islands',
  'mimeType': 'application/vnd.google-apps.folder'},
 {'kind': 'drive#file',
  'id': '1NeT6s5LznLXYTlyY1Z-jLuTGXFzQVLOk',
  'name': 'Utah',
  'mimeType': 'application/vnd.google-apps

In [76]:
# Find all CSV files in the child folders
data_subfolder_ids = []

# Get the subfolder IDs
for subfolder in data_subfolders:
    data_subfolder_ids.append(subfolder['id'])
    
data_subfolder_ids

['1g92YGs6BD40S89CqRZhlL2XXPQE-JIR8',
 '19OOYSCuzi2jGJXUBMb8EDtfUxjRW8NyN',
 '1LCRqy05NV8nwaOg-YJOnt0ueow0WFJNq',
 '1XVK0rzn9DteU5Jg5rMootujPKzyfP5bS',
 '1NJNG2ewGUrela4JCNZOeq-rCs5WNX-RT',
 '1rUua0BiZFioslEY-LJJMXglGIdFMzGly',
 '1NeT6s5LznLXYTlyY1Z-jLuTGXFzQVLOk',
 '1y9S9t8zVdazAjiqQQJMvC6vJcpqn940e',
 '1ujNT5X0T8HyXWIgSgmyOACqLjb7lIRfz',
 '1qlt355n6bPlpXgb0e65PVwliRiPQX_Z1',
 '1Gea62RaeWCFVRFv0SdNtTE1URaU1YCPN',
 '1xbGhYS-kfBp2sePhquDU8fy7Wy8Vm8Kn',
 '1lMJJjNh_MKXRWpkyt4YDsyJfPBu06hNM',
 '1tYblyYoqg-ggsHvrjTvJ2ntFiMr0pY4M',
 '11iQjkDi6HT2YengU3beDzgbuHZk9LQhj',
 '12vf4_vOxS3HScnP5Ba9GMoiGECLuKD7A',
 '17gkIBQe2eAcCkRKeJ0FSxvQGVFsEmpqU',
 '1s2nsIS6j7Vt2pdiTmFk7O5EI0ybRosqM',
 '1TSry8Gw8yy9JhWTX2W8sqRiYUd3MjdM-',
 '1r1NnJlUJmDVhX0ecdxfizr9BwbZH0rI5',
 '1e2hVMMlyWHs1podHUsA2sbyrK8TGv-B4',
 '121M-DDixDLNXy43bu-dKNpIHR8pEDKas',
 '1UR8CofrtH8EFMPSf7chIlSKjkCxWSacY',
 '1waA32WjdtBktdDUi4ECcbDHae2zSGeog',
 '1VfG6425yIcUKXhJqG-DuooheYmGd_l_-',
 '1CioK5rO3ytUg82ViytlegaNpGy-8UqyW',
 '1q3ktB6faZ

In [86]:
query = " in parents or ".join([f"'{folder_id}'" for folder_id in data_subfolder_ids])
query += " in parents"
query = f"({query})"
query +=  "and mimeType = 'text/csv'"

In [ ]:
# Get all CSV file IDs from data subfolders as a list of ByteStrings
data_file_ids = service.files().list(
q = query).execute().get('files', []) 


# Pull and concatenate all ByteStrings, skipping headers, 
# and decode into single DataFrame for further analysis
# Note that this would be more efficient long-term to ZIP all CSVs into one file 
# and then pull that down alone
for i, file in enumerate(data_file_ids):
    if i == 0:
        data_str = service.files()\
        .get_media(fileId=file['id'])\
        .execute()
        
    # just concatenating here, without header, since we already have it
    else:
        temp_data_str = service.files()\
        .get_media(fileId=file['id'])\
        .execute()
        
        # Assuming here that header is the same across files and thus we can skip it
        # Find end of header by finding first newline character
        data_start_index = temp_data_str.find(b"\n") + 1

        data_str += temp_data_str[data_start_index:]
        # Check that \r\n is at end of string, add it if not
        if data_str[-2:] != b'\r\n':
            data_str += b'\r\n'

# Decode ByteString into something that pandas can make a DataFrame out of
data = data_str.decode('utf8').encode('latin-1')
df = pd.read_csv(BytesIO(data), encoding='latin-1', low_memory=False)

df.info(memory_usage='deep')

### All Together Now!

In [2]:
%%time
from src.data.make_dataset import pull_ppp_data

df = pull_ppp_data(local_copy='/home/jovyan/work/data/raw/unprocessed_ppp_data.csv')
df.info(memory_usage='deep')

100%|██████████| 58/58 [02:23<00:00,  2.47s/it]
/opt/conda/envs/CARES_Act/lib/python3.7/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (0,3,4,5,6,10,12,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4885388 entries, 0 to 4885387
Data columns (total 16 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   LoanRange      object
 1   BusinessName   object
 2   Address        object
 3   City           object
 4   State          object
 5   Zip            object
 6   NAICSCode      object
 7   BusinessType   object
 8   RaceEthnicity  object
 9   Gender         object
 10  Veteran        object
 11  NonProfit      object
 12  JobsRetained   object
 13  DateApproved   object
 14  Lender         object
 15  CD             object
dtypes: object(16)
memory usage: 596.4+ MB
CPU times: user 2min 55s, sys: 55.3 s, total: 3min 50s
Wall time: 8min 14s


In [5]:
%%time
# If you've already saved a local copy...
df = pd.read_csv('data/raw/unprocessed_ppp_data.csv')
df.info(memory_usage='deep')

<string>:2: DtypeWarning: Columns (0,3,4,5,6,10,12,14,15) have mixed types.Specify dtype option on import or set low_memory=False.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4885388 entries, 0 to 4885387
Data columns (total 16 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   LoanRange      object
 1   BusinessName   object
 2   Address        object
 3   City           object
 4   State          object
 5   Zip            object
 6   NAICSCode      object
 7   BusinessType   object
 8   RaceEthnicity  object
 9   Gender         object
 10  Veteran        object
 11  NonProfit      object
 12  JobsRetained   object
 13  DateApproved   object
 14  Lender         object
 15  CD             object
dtypes: object(16)
memory usage: 3.9 GB
CPU times: user 1min 40s, sys: 12.3 s, total: 1min 52s
Wall time: 2min 6s


## Determining Patterns to Bad Data

There are [a number of documented issues](https://qz.com/1878225/heres-what-we-know-is-wrong-with-the-ppp-data/) with the PPP dataset and I have a sneaking suspicion that the problems correlate with lender name to a large extent (e.g. a single lender tended to put ZIP codes in the State section of the data). Let's see if that plays out at all.

In [4]:
# First memory_usage=es = 87,689 CA counts as top
df['State'].value_counts()

722511.0    135496
531210.0    110875
541110.0    107600
621210.0     88677
CA           87689
             ...  
335220           1
813940           1
313220           1
233110           1
315233.0         1
Name: State, Length: 2170, dtype: int64

In [6]:
# Is there any obvious pattern to the (loan amount?) being in the State field? E.g. by lender?
df[df['State'] == 722511.0]

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsRetained,DateApproved,Lender,CD
688136,135913,CITY OF INDUSTRY,CA,91748,722511,Corporation,Unanswered,Unanswered,Unanswered,NaN,33,05/17/2020,"Bank of America, National Association",CA - 32,NaN,NaN
688157,135900,SAN FRANCISCO,CA,94107,722511,Corporation,Unanswered,Unanswered,Unanswered,NaN,NaN,06/09/2020,GBC International Bank,CA - 01,NaN,NaN
688175,135870,SAN DIEGO,CA,92111,722511,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,10,04/14/2020,"JPMorgan Chase Bank, National Association",CA - 52,NaN,NaN
688176,135870,OAKLAND,CA,94611,722511,Corporation,Unanswered,Unanswered,Unanswered,NaN,11,05/03/2020,"U.S. Bank, National Association",CA - 11,NaN,NaN
688201,135800,EL CENTRO,CA,92243,722511,Corporation,Hispanic,Male Owned,Non-Veteran,NaN,14,04/10/2020,Community Valley Bank,CA - 51,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4885092,500,MONTEZUMA,GA,31063,722511,Partnership,Unanswered,Unanswered,Unanswered,NaN,NaN,06/26/2020,Truist Bank d/b/a Branch Banking & Trust Co,GA - 02,NaN,NaN
4885160,421,DULUTH,GA,30096,722511,Corporation,Unanswered,Unanswered,Unanswered,NaN,0,05/08/2020,"Wells Fargo Bank, National Association",GA - 07,NaN,NaN
4885251,260,DEARING,GA,30808,722511,Sole Proprietorship,Unanswered,Unanswered,Unanswered,NaN,0,05/07/2020,Regions Bank,GA - 10,NaN,NaN
4885312,133,SUWANEE,GA,30024,722511,Corporation,Unanswered,Unanswered,Unanswered,NaN,3,05/08/2020,"Bank of America, National Association",GA - 06,NaN,NaN


In [18]:
# What is that NAICS code?
naics = pd.read_csv('../data/Lookup Tables/NAICS Codes.csv',
                    usecols=['NAICS', 'Industry'],
                   encoding='latin-1')
naics.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2196 entries, 0 to 2195
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   NAICS     2196 non-null   object
 1   Industry  2196 non-null   object
dtypes: object(2)
memory usage: 336.4 KB


In [21]:
naics[naics['NAICS'] == '722511']

,NAICS,Industry
2019,722511,Full-Service Restaurants


In [22]:
# Consistent across lenders maybe?
df.loc[df['State'] == 722511.0, 'JobsRetained'].value_counts()

count    4062.000000
mean       33.356967
std       234.258549
min         1.000000
25%         3.000000
50%         7.000000
75%        17.000000
max      9562.000000
Name: JobsRetained, dtype: float64

In [39]:
lender_value_counts = df.loc[df['State'] == 722511.0, 'JobsRetained'].value_counts()

lender_value_counts[lender_value_counts > 100]

Bank of America, National Association          9562
JPMorgan Chase Bank, National Association      5852
Wells Fargo Bank, National Association         5820
Truist Bank d/b/a Branch Banking & Trust Co    2895
U.S. Bank, National Association                2588
                                               ... 
ConnectOne Bank                                 105
Premier Bank                                    105
Dime Community Bank                             105
1st Source Bank                                 102
Heartland Bank and Trust Company                102
Name: JobsRetained, Length: 177, dtype: int64

In [41]:
lender_value_counts[lender_value_counts > 1000]

Bank of America, National Association          9562
JPMorgan Chase Bank, National Association      5852
Wells Fargo Bank, National Association         5820
Truist Bank d/b/a Branch Banking & Trust Co    2895
U.S. Bank, National Association                2588
Kabbage, Inc.                                  2437
TD Bank, National Association                  2261
Cross River Bank                               2234
Celtic Bank Corporation                        2217
KeyBank National Association                   1894
Citizens Bank, National Association            1604
WebBank                                        1565
PNC Bank, National Association                 1480
The Huntington National Bank                   1404
Customers Bank                                 1284
Regions Bank                                   1120
Fifth Third Bank                               1023
Name: JobsRetained, dtype: int64

In [ ]:
px.bar()

This appears to have every entry offset by two columns to the left (e.g. `City` entry appears in the `BusinessName` column. As such, the value we filtered by (`df['State'] == 722511.0`) is actually the NAICS code, which corresponds to "Full-Service Restaurants".

Oddly enough, **`LoanRange`** seems to be correct (as it *should* be `Address` value if the shifting of columns was kept consistent). Not sure why.